In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

from keras import backend as K

# Бэкенд — это термин в Keras, который выполняет все низкоуровневые вычисления,
# такие как тензорные произведения, свертки и многое другое, с помощью других библиотек, таких как Tensorflow или Theano.

from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPooling2D  # , GlobalAveragePooling2D
from keras.layers import Activation, Dropout, BatchNormalization, Flatten, Dense
# from keras.layers import AvgPool2D, MaxPool2D
# from keras.applications.vgg16 import VGG16, preprocess_input
from keras.optimizers import Adam, SGD, RMSprop

import tensorflow as tf

from PIL import Image, ImageDraw

import numpy as np
import pandas as pd
import random

import matplotlib.pyplot as plt
%matplotlib inline

Подключим диск, на который предварительно были загружены предобработанные данные.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import json

d_train = dict()
with open('drive/MyDrive/train_data/ChestX_Det_train.json', 'r', encoding='utf-8') as f:
    reader = json.load(f)
    for r in reader:
        if set(r['syms']) == set():
            d_train[r['file_name']] = {'syms': ('Normal',), 'boxes': list()}
        else:
            cat = tuple(set(r['syms']))
            d_train[r['file_name']] = {'syms': cat, 'boxes': list()}
            for i in range(len(cat)):
                d_train[r['file_name']]['boxes'].append(r['boxes'][i])

In [ ]:
h = 0
w = 0
for v in d_train.values():
    if v['boxes']:
        for box in v['boxes']:
            a = abs(box[0] - box[2])
            b = abs(box[1] - box[3])
            if a > w:
                w = a
            if b > h:
                h = b

In [ ]:
h, w

(915, 603)

Общее количество классов, на которые делятся все изображения.

In [ ]:
NB_CLASSES = 13 + 1

Как оптимально подобрать количество эпох обучения и параметр `BATCH_SIZE`?

Как выбираем <a href="https://stackoverflow.com/questions/56320749/how-to-choose-batch-size-steps-per-epoch-and-epoch-with-keras-generator">BATCH_SIZE</a>?

In [ ]:
BATCH_SIZE = 32

Как выбираем EPOCHS?

Предварительно установим значение 8.

In [ ]:
EPOCHS = 8

Подготовка изображений для обучения.

О параметрах конструктора класса <a href="https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator">ImageDataGenerator</a>:
- **shear_range** позволяет случайным образом искривлять изображения путем изменения угла наклона объектов на изображении: каждая точка изображения перемещается вдоль одной из осей. Изменение угла наклона может быть полезным, если данные содержат объекты, наклоненные под разными углами, с целью обучения модели на изображениях с различными наклонами.
- **zoom_range** случайным образом масштабирует изображения.
- **horizontal_flip** случайным образом горизонтально отражает входные данные.

<a href="https://www.educba.com/keras-imagedatagenerator/">Туториал</a> по использованию этого класса.

In [ ]:
datagen = ImageDataGenerator(
    rescale=1/255,
    rotation_range=20,
    width_shift_range=.2,
    height_shift_range=.2,
    shear_range=.2,
    zoom_range=.2,
    horizontal_flip=True
)

О методе flow_from_directory.

Метод **flow_from_directory** класса ImageDataGenerator предоставляет возможность генерировать пакеты данных (batch) из изображений, хранящихся в определенной директории. Вместо загрузки всех изображений в память сразу, этот метод позволяет загружать данные порциями (пакетами) по мере необходимости.

Реализуем генераторные функции для обучения.

<a href="https://vijayabhaskar96.medium.com/tutorial-image-classification-with-keras-flow-from-directory-and-generators-95f75ebe5720">Подробнее</a> про метод flow_from_directory.

In [ ]:
train_directory = 'drive/MyDrive/train_data/train'
val_directory = 'drive/MyDrive/val_data/val'
test_directory = 'drive/MyDrive/test_data/test'

In [ ]:
train_generator = datagen.flow_from_directory(
    directory=train_directory,
    target_size=(512, 512),
    color_mode='grayscale',
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True,
    seed=0
)

Found 4570 images belonging to 14 classes.


In [ ]:
validation_generator = datagen.flow_from_directory(
    directory=val_directory,
    target_size=(512, 512),
    color_mode='grayscale',
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True,
    seed=0
)

Found 1142 images belonging to 14 classes.


In [ ]:
test_generator = datagen.flow_from_directory(
    directory=test_directory,
    target_size=(512, 512),
    color_mode='grayscale',
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True,
    seed=0
)

Found 1236 images belonging to 14 classes.


Спроектируем архитектуру свёрточной НС.

In [ ]:
cnn = Sequential([
    Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=(512, 512, 1)),
    MaxPooling2D((2, 2), strides=2),
    Conv2D(64, (3, 3), padding='same', activation='relu'),
    MaxPooling2D((2, 2), strides=2),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(NB_CLASSES, activation='softmax')
])

In [ ]:
cnn.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy'])

In [ ]:
cnn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 512, 512, 32)      320       
                                                                 
 max_pooling2d (MaxPooling2  (None, 256, 256, 32)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 256, 256, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 128, 128, 64)      0         
 g2D)                                                            
                                                                 
 flatten (Flatten)           (None, 1048576)           0         
                                                                 
 dense (Dense)               (None, 128)               1

In [ ]:
cnn_history = cnn.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=validation_generator
)

Epoch 1/8
143/143 [==============================] - 2247s 16s/step - loss: 5.1960 - accuracy: 0.2672 - val_loss: 2.1358 - val_accuracy: 0.3152
Epoch 2/8
143/143 [==============================] - 189s 1s/step - loss: 2.0898 - accuracy: 0.3221 - val_loss: 2.3208 - val_accuracy: 0.2504
Epoch 3/8
143/143 [==============================] - 188s 1s/step - loss: 2.2002 - accuracy: 0.2932 - val_loss: 2.1280 - val_accuracy: 0.2872
Epoch 4/8
143/143 [==============================] - 188s 1s/step - loss: 2.0846 - accuracy: 0.3059 - val_loss: 2.0371 - val_accuracy: 0.3126
Epoch 5/8
143/143 [==============================] - 188s 1s/step - loss: 2.0286 - accuracy: 0.3199 - val_loss: 1.9870 - val_accuracy: 0.3205
Epoch 6/8
143/143 [==============================] - 189s 1s/step - loss: 2.0086 - accuracy: 0.3348 - val_loss: 1.9713 - val_accuracy: 0.3450
Epoch 7/8
143/143 [==============================] - 194s 1s/step - loss: 1.9851 - accuracy: 0.3442 - val_loss: 2.0097 - val_accuracy: 0.3117
Epoc

In [ ]:
cnn.evaluate(
      test_generator
)

39/39 [==============================] - 562s 15s/step - loss: 2.1544 - accuracy: 0.2880


[2.154402732849121, 0.2880258858203888]

Попробуем обучить сеть с архитектурой VGG-16.


In [ ]:
vgg = Sequential([
    Conv2D(16, (3, 3), padding='same', activation='relu', input_shape=(512, 512, 1)),
    Conv2D(16, (3, 3), padding='same', activation='relu'),
    MaxPooling2D((2, 2), strides=2),
    Conv2D(32, (3, 3), padding='same', activation='relu'),
    Conv2D(32, (3, 3), padding='same', activation='relu'),
    MaxPooling2D((2, 2), strides=2),
    Conv2D(64, (3, 3), padding='same', activation='relu'),
    Conv2D(64, (3, 3), padding='same', activation='relu'),
    Conv2D(64, (3, 3), padding='same', activation='relu'),
    MaxPooling2D((2, 2), strides=2),
    Conv2D(128, (3, 3), padding='same', activation='relu'),
    Conv2D(128, (3, 3), padding='same', activation='relu'),
    Conv2D(128, (3, 3), padding='same', activation='relu'),
    MaxPooling2D((2, 2), strides=2),
    Conv2D(128, (3, 3), padding='same', activation='relu'),
    Conv2D(128, (3, 3), padding='same', activation='relu'),
    Conv2D(128, (3, 3), padding='same', activation='relu'),
    MaxPooling2D((2, 2), strides=2),
    Flatten(),
    Dense(256, activation='relu'),
    Dense(256, activation='relu'),
    Dense(NB_CLASSES, activation='softmax')
])

In [ ]:
vgg.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy'])

In [ ]:
vgg.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_15 (Conv2D)          (None, 512, 512, 16)      160       
                                                                 
 conv2d_16 (Conv2D)          (None, 512, 512, 16)      2320      
                                                                 
 max_pooling2d_7 (MaxPoolin  (None, 256, 256, 16)      0         
 g2D)                                                            
                                                                 
 conv2d_17 (Conv2D)          (None, 256, 256, 32)      4640      
                                                                 
 conv2d_18 (Conv2D)          (None, 256, 256, 32)      9248      
                                                                 
 max_pooling2d_8 (MaxPoolin  (None, 128, 128, 32)      0         
 g2D)                                                 

In [ ]:
vgg_history = vgg.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=validation_generator
)

Epoch 1/8
143/143 [==============================] - 225s 1s/step - loss: 2.3574 - accuracy: 0.2302 - val_loss: 2.2845 - val_accuracy: 0.2356
Epoch 2/8
143/143 [==============================] - 197s 1s/step - loss: 2.3334 - accuracy: 0.2324 - val_loss: 2.2912 - val_accuracy: 0.2356
Epoch 3/8
143/143 [==============================] - 200s 1s/step - loss: 2.3321 - accuracy: 0.2333 - val_loss: 2.2912 - val_accuracy: 0.2356
Epoch 4/8
143/143 [==============================] - 197s 1s/step - loss: 2.3303 - accuracy: 0.2339 - val_loss: 2.2978 - val_accuracy: 0.2356
Epoch 5/8
143/143 [==============================] - 195s 1s/step - loss: 2.3268 - accuracy: 0.2317 - val_loss: 2.3025 - val_accuracy: 0.2356
Epoch 6/8
143/143 [==============================] - 196s 1s/step - loss: 2.3252 - accuracy: 0.2339 - val_loss: 2.2796 - val_accuracy: 0.2356
Epoch 7/8
143/143 [==============================] - 196s 1s/step - loss: 2.3251 - accuracy: 0.2337 - val_loss: 2.2814 - val_accuracy: 0.2356
Epoch 

In [ ]:
datagen = ImageDataGenerator(
    rescale=1/255,
    rotation_range=20,
    width_shift_range=.2,
    height_shift_range=.2,
    shear_range=.2,
    zoom_range=.2,
    horizontal_flip=True
)

In [ ]:
model_one_directory = 'drive/MyDrive/model one'
model_two_directory = 'drive/MyDrive/model two'

In [ ]:
model_one_train_generator = datagen.flow_from_directory(
    directory=model_one_directory,
    target_size=(512, 512),
    color_mode='grayscale',
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True,
    seed=0
)

Found 2530 images belonging to 8 classes.


Папа, запусти, пожалуйста, ячейку ниже.

In [ ]:
model_two_train_generator = datagen.flow_from_directory(
    directory=model_two_directory,
    target_size=(512, 512),
    color_mode='grayscale',
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True,
    seed=0
)

Found 2738 images belonging to 8 classes.


In [ ]:
classes = 8
model_one_cnn = Sequential([
    Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=(512, 512, 1)),
    MaxPooling2D((2, 2), strides=2),
    Conv2D(64, (3, 3), padding='same', activation='relu'),
    MaxPooling2D((2, 2), strides=2),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(classes, activation='softmax')
])

In [ ]:
model_one_cnn.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy'])

In [ ]:
model_one_cnn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 512, 512, 32)      320       
                                                                 
 max_pooling2d (MaxPooling2  (None, 256, 256, 32)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 256, 256, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 128, 128, 64)      0         
 g2D)                                                            
                                                                 
 flatten (Flatten)           (None, 1048576)           0         
                                                                 
 dense (Dense)               (None, 128)               1

In [ ]:
model_one_history = model_one_cnn.fit(
    model_one_train_generator,
    epochs=EPOCHS+8
)

Epoch 1/16
80/80 [==============================] - 588s 7s/step - loss: 6.7632 - accuracy: 0.3968
Epoch 2/16
80/80 [==============================] - 82s 1s/step - loss: 1.7206 - accuracy: 0.4328
Epoch 3/16
80/80 [==============================] - 84s 1s/step - loss: 1.7333 - accuracy: 0.4336
Epoch 4/16
80/80 [==============================] - 82s 1s/step - loss: 1.7270 - accuracy: 0.4320
Epoch 5/16
80/80 [==============================] - 84s 1s/step - loss: 1.6969 - accuracy: 0.4324
Epoch 6/16
80/80 [==============================] - 82s 1s/step - loss: 1.6634 - accuracy: 0.4403
Epoch 7/16
80/80 [==============================] - 84s 1s/step - loss: 1.6753 - accuracy: 0.4379
Epoch 8/16
80/80 [==============================] - 82s 1s/step - loss: 1.6308 - accuracy: 0.4411
Epoch 9/16
80/80 [==============================] - 84s 1s/step - loss: 1.6227 - accuracy: 0.4411
Epoch 10/16
80/80 [==============================] - 82s 1s/step - loss: 1.6319 - accuracy: 0.4411
Epoch 11/16
80/80 

In [ ]:
classes = 8
model_two_cnn = Sequential([
    Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=(512, 512, 1)),
    MaxPooling2D((2, 2), strides=2),
    Conv2D(64, (3, 3), padding='same', activation='relu'),
    MaxPooling2D((2, 2), strides=2),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(classes, activation='softmax')
])

In [ ]:
model_two_cnn.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy'])

In [ ]:
model_two_cnn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 512, 512, 32)      320       
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 256, 256, 32)      0         
 g2D)                                                            
                                                                 
 conv2d_3 (Conv2D)           (None, 256, 256, 64)      18496     
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 128, 128, 64)      0         
 g2D)                                                            
                                                                 
 flatten_1 (Flatten)         (None, 1048576)           0         
                                                                 
 dense_1 (Dense)             (None, 128)               1

In [ ]:
model_two_history = model_two_cnn.fit(
    model_two_train_generator,
    epochs=EPOCHS+8
)

Epoch 1/16
86/86 [==============================] - 1593s 18s/step - loss: 9.9717 - accuracy: 0.3378
Epoch 2/16
86/86 [==============================] - 1428s 17s/step - loss: 1.7895 - accuracy: 0.4226
Epoch 3/16
86/86 [==============================] - 1445s 17s/step - loss: 1.5639 - accuracy: 0.4638
Epoch 4/16
45/86 [==============>...............] - ETA: 12:37 - loss: 1.4702 - accuracy: 0.4958

In [ ]:
simple_cnn = Sequential([
    Conv2D(
        filters=32,
        kernel_size=(5, 5),
        activation='relu',
        input_shape=(512, 512, 1),
        padding='same'
    ),
    MaxPooling2D(2, 2),
    Conv2D(
        filters=64,
        kernel_size=(3, 3),
        activation='relu',
        padding='same'
    ),
    MaxPooling2D(2, 2),
    Conv2D(
        filters=128,
        kernel_size=(3, 3),
        activation='relu',
        padding='same'
      ),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(256, activation='relu'),
    BatchNormalization(),
    Dense(128, activation='relu'),
    Dropout(0.3),
    BatchNormalization(),
    Dense(8, activation='softmax')
])

In [ ]:
simple_cnn.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy'])

In [ ]:
simple_cnn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 512, 512, 32)      832       
                                                                 
 max_pooling2d (MaxPooling2  (None, 256, 256, 32)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 256, 256, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 128, 128, 64)      0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 128, 128, 128)     73856     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 64, 64, 128)       0

In [ ]:
simple_cnn_history = simple_cnn.fit(
    model_one_train_generator,
    epochs=EPOCHS+8
)

Epoch 1/16
80/80 [==============================] - 711s 9s/step - loss: 2.1630 - accuracy: 0.2613
Epoch 2/16
80/80 [==============================] - 90s 1s/step - loss: 1.9065 - accuracy: 0.3514
Epoch 3/16
80/80 [==============================] - 87s 1s/step - loss: 1.8259 - accuracy: 0.3909
Epoch 4/16
80/80 [==============================] - 90s 1s/step - loss: 1.7641 - accuracy: 0.3897
Epoch 5/16
80/80 [==============================] - 88s 1s/step - loss: 1.7437 - accuracy: 0.4107
Epoch 6/16
80/80 [==============================] - 87s 1s/step - loss: 1.7149 - accuracy: 0.4206
Epoch 7/16
80/80 [==============================] - 89s 1s/step - loss: 1.7210 - accuracy: 0.4170
Epoch 8/16
80/80 [==============================] - 90s 1s/step - loss: 1.6750 - accuracy: 0.4320
Epoch 9/16
80/80 [==============================] - 100s 1s/step - loss: 1.6510 - accuracy: 0.4340
Epoch 10/16
80/80 [==============================] - 90s 1s/step - loss: 1.6618 - accuracy: 0.4316
Epoch 11/16
80/80

In [ ]:
simple_cnn_two = Sequential([
    Conv2D(
        filters=32,
        kernel_size=(5, 5),
        activation='relu',
        input_shape=(512, 512, 1),
        padding='same'
    ),
    MaxPooling2D(2, 2),
    Conv2D(
        filters=64,
        kernel_size=(3, 3),
        activation='relu',
        padding='same'
    ),
    MaxPooling2D(2, 2),
    Conv2D(
        filters=128,
        kernel_size=(3, 3),
        activation='relu',
        padding='same'
      ),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(256, activation='relu'),
    BatchNormalization(),
    Dense(128, activation='relu'),
    Dropout(0.3),
    BatchNormalization(),
    Dense(8, activation='softmax')
])

In [ ]:
simple_cnn_two.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy'])

In [ ]:
simple_cnn_two.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 512, 512, 32)      832       
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 256, 256, 32)      0         
 g2D)                                                            
                                                                 
 conv2d_4 (Conv2D)           (None, 256, 256, 64)      18496     
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 128, 128, 64)      0         
 g2D)                                                            
                                                                 
 conv2d_5 (Conv2D)           (None, 128, 128, 128)     73856     
                                                                 
 max_pooling2d_5 (MaxPoolin  (None, 64, 64, 128)      

In [ ]:
simple_cnn_two_history = simple_cnn_two.fit(
    model_two_train_generator,
    epochs=EPOCHS+8
)

Epoch 1/16
86/86 [==============================] - 825s 10s/step - loss: 1.6891 - accuracy: 0.4043
Epoch 2/16
86/86 [==============================] - 93s 1s/step - loss: 1.5403 - accuracy: 0.4511
Epoch 3/16
86/86 [==============================] - 95s 1s/step - loss: 1.5230 - accuracy: 0.4576
Epoch 4/16
86/86 [==============================] - 94s 1s/step - loss: 1.5004 - accuracy: 0.4763
Epoch 5/16
86/86 [==============================] - 93s 1s/step - loss: 1.4687 - accuracy: 0.4690
Epoch 6/16
86/86 [==============================] - 95s 1s/step - loss: 1.4430 - accuracy: 0.4803
Epoch 7/16
86/86 [==============================] - 95s 1s/step - loss: 1.4536 - accuracy: 0.4795
Epoch 8/16
86/86 [==============================] - 95s 1s/step - loss: 1.4086 - accuracy: 0.4898
Epoch 9/16
86/86 [==============================] - 95s 1s/step - loss: 1.4137 - accuracy: 0.4909
Epoch 10/16
86/86 [==============================] - 94s 1s/step - loss: 1.4110 - accuracy: 0.4931
Epoch 11/16
86/86

In [ ]:
my_cnn = Sequential([
    Conv2D(
        filters=32,
        kernel_size=(5, 5),
        activation='relu',
        input_shape=(512, 512, 1),
        padding='same'
    ),
    MaxPooling2D(2, 2),
    Conv2D(
        filters=64,
        kernel_size=(4, 4),
        activation='relu',
        padding='same'
    ),
    MaxPooling2D(2, 2),
    Conv2D(
        filters=128,
        kernel_size=(3, 3),
        activation='relu',
        padding='same'
      ),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(256, activation='relu'),
    BatchNormalization(),
    Dense(128, activation='relu'),
    BatchNormalization(),
    Dense(64, activation='relu'),
    Dense(8, activation='softmax')
])

In [ ]:
my_cnn.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy'])

In [ ]:
my_cnn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 512, 512, 32)      832       
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 256, 256, 32)      0         
 g2D)                                                            
                                                                 
 conv2d_4 (Conv2D)           (None, 256, 256, 64)      32832     
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 128, 128, 64)      0         
 g2D)                                                            
                                                                 
 conv2d_5 (Conv2D)           (None, 128, 128, 128)     73856     
                                                                 
 max_pooling2d_5 (MaxPoolin  (None, 64, 64, 128)      

In [ ]:
my_cnn_history = my_cnn.fit(
    model_one_train_generator,
    epochs=EPOCHS+8
)

Epoch 1/16
80/80 [==============================] - 2034s 25s/step - loss: 1.7306 - accuracy: 0.4016
Epoch 2/16
80/80 [==============================] - 1993s 25s/step - loss: 1.6284 - accuracy: 0.4399
Epoch 3/16
80/80 [==============================] - 2005s 25s/step - loss: 1.6261 - accuracy: 0.4451
Epoch 4/16
80/80 [==============================] - 1982s 25s/step - loss: 1.6234 - accuracy: 0.4451
Epoch 5/16
80/80 [==============================] - 1975s 25s/step - loss: 1.5951 - accuracy: 0.4522
Epoch 6/16
80/80 [==============================] - 1967s 25s/step - loss: 1.5977 - accuracy: 0.4498
Epoch 7/16
80/80 [==============================] - 1960s 24s/step - loss: 1.6066 - accuracy: 0.4573
Epoch 8/16
80/80 [==============================] - 1966s 25s/step - loss: 1.6089 - accuracy: 0.4538
Epoch 9/16
80/80 [==============================] - 1957s 24s/step - loss: 1.5914 - accuracy: 0.4530
Epoch 10/16
80/80 [==============================] - 1967s 25s/step - loss: 1.5882 - accura

In [ ]:
my_cnn_two = Sequential([
    Conv2D(
        filters=32,
        kernel_size=(5, 5),
        activation='relu',
        input_shape=(512, 512, 1),
        padding='same'
    ),
    MaxPooling2D(2, 2),
    Conv2D(
        filters=64,
        kernel_size=(4, 4),
        activation='relu',
        padding='same'
    ),
    MaxPooling2D(2, 2),
    Conv2D(
        filters=128,
        kernel_size=(3, 3),
        activation='relu',
        padding='same'
      ),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(256, activation='relu'),
    BatchNormalization(),
    Dense(128, activation='relu'),
    BatchNormalization(),
    Dense(64, activation='relu'),
    Dropout(0.3),
    BatchNormalization(),
    Dense(8, activation='softmax')
])

In [ ]:
my_cnn_two.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy'])

In [ ]:
my_cnn_two.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 512, 512, 32)      832       
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 256, 256, 32)      0         
 g2D)                                                            
                                                                 
 conv2d_4 (Conv2D)           (None, 256, 256, 64)      32832     
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 128, 128, 64)      0         
 g2D)                                                            
                                                                 
 conv2d_5 (Conv2D)           (None, 128, 128, 128)     73856     
                                                                 
 max_pooling2d_5 (MaxPoolin  (None, 64, 64, 128)      

In [ ]:
my_cnn_two_history = my_cnn_two.fit(
    model_two_train_generator,
    epochs=EPOCHS+8
)

Epoch 1/16
42/86 [=============>................] - ETA: 20:23 - loss: 1.9454 - accuracy: 0.3143